## Carga de datos

In [ ]:
%%capture
!pip install mlflow

In [ ]:
import mlflow
import os

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

In [ ]:
%%capture
!pip install kagglehub[pandas-dataset]
!pip install optuna
!pip install keras
!pip install pyngrok

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import itertools
import kagglehub
from kagglehub import KaggleDatasetAdapter
import optuna
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import keras
from sklearn.metrics import classification_report, accuracy_score
import mlflow
import os
from pyngrok import ngrok


from sklearn.model_selection import train_test_split

In [ ]:
SEED = 7
token = "2zbl0dEaBIwHYbdy8mg0wAmtd7z_3LPkpSEY88zSz53BdUUbg" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
ngrok.connect(5000, "http")
mlflow.set_tracking_uri("http://localhost:5000")
exp_id = mlflow.create_experiment(name="Prediction_addicted", artifact_location="mlruns/")

In [ ]:
file_path = "Students Social Media Addiction.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "adilshamim8/social-media-addiction-vs-relationships",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("Columns:", df.columns)

/tmp/ipython-input-9-1368852353.py:4: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Columns: Index(['Student_ID', 'Age', 'Gender', 'Academic_Level', 'Country',
       'Avg_Daily_Usage_Hours', 'Most_Used_Platform',
       'Affects_Academic_Performance', 'Sleep_Hours_Per_Night',
       'Mental_Health_Score', 'Relationship_Status',
       'Conflicts_Over_Social_Media', 'Addicted_Score'],
      dtype='object')


## Preprocesamiento

In [ ]:
df = df.drop(columns=['Student_ID']).drop_duplicates()
X=df.drop(columns=['Addicted_Score'])
y=df['Addicted_Score']

In [ ]:
X['Academic_Level'].unique()
y.unique()

array([8, 3, 9, 4, 7, 2, 5, 6])

In [ ]:
#Mapping academic levels
X["Academic_Level"] = X["Academic_Level"].replace({
    "Undergraduate": 2,
    "Graduate": 3,
    "High School": 1
})

/tmp/ipython-input-12-3165889170.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X["Academic_Level"] = X["Academic_Level"].replace({


In [ ]:
X_cat = X.select_dtypes(include=['object'])
X_num = X.select_dtypes(exclude=['object'])

In [ ]:
X_cat_encoded = pd.get_dummies(X_cat, drop_first=True)
X_cat_encoded

,Gender_Male,Country_Albania,Country_Andorra,Country_Argentina,Country_Armenia,Country_Australia,Country_Austria,Country_Azerbaijan,Country_Bahamas,Country_Bahrain,...,Most_Used_Platform_Snapchat,Most_Used_Platform_TikTok,Most_Used_Platform_Twitter,Most_Used_Platform_VKontakte,Most_Used_Platform_WeChat,Most_Used_Platform_WhatsApp,Most_Used_Platform_YouTube,Affects_Academic_Performance_Yes,Relationship_Status_In Relationship,Relationship_Status_Single
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
701,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
702,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,True,False
703,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [ ]:
X_final = pd.concat([X_num, X_cat_encoded], axis=1)
X_final

,Age,Academic_Level,Avg_Daily_Usage_Hours,Sleep_Hours_Per_Night,Mental_Health_Score,Conflicts_Over_Social_Media,Gender_Male,Country_Albania,Country_Andorra,Country_Argentina,...,Most_Used_Platform_Snapchat,Most_Used_Platform_TikTok,Most_Used_Platform_Twitter,Most_Used_Platform_VKontakte,Most_Used_Platform_WeChat,Most_Used_Platform_WhatsApp,Most_Used_Platform_YouTube,Affects_Academic_Performance_Yes,Relationship_Status_In Relationship,Relationship_Status_Single
0,19,2,5.2,6.5,6,3,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,22,3,2.1,7.5,8,0,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,20,2,6.0,5.0,5,4,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
3,18,1,3.0,7.0,7,1,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,21,3,4.5,6.0,6,2,True,False,False,False,...,False,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,20,2,4.7,7.2,7,2,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
701,23,3,6.8,5.9,4,5,True,False,False,False,...,False,False,False,False,False,False,False,True,False,True
702,21,2,5.6,6.7,6,3,False,False,False,False,...,False,False,False,False,True,False,False,True,True,False
703,24,3,4.3,7.5,8,2,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [ ]:
x_test , x_train , y_test , y_train  = train_test_split(X_final , y, test_size=0.7, random_state=7)
x_val , x_test , y_val , y_test  = train_test_split( x_test , y_test , test_size=0.5, random_state=7)

In [ ]:
x_train.shape , y_train.shape , x_test.shape , y_test.shape , x_val.shape , y_val.shape

((493, 130), (493,), (106, 130), (106,), (105, 130), (105,))

Finalmente, intentamos con una red neuronal sencilla, para esto debemos convertir las salidas con one hot encode

In [ ]:
y_train_encoded = pd.get_dummies(y_train, drop_first=False)
y_val_encoded = pd.get_dummies(y_val, drop_first=False)
y_test_encoded = pd.get_dummies(y_test, drop_first=False)

print("Shape of y_train after one-hot encoding:", y_train_encoded.shape)
print("Shape of y_val after one-hot encoding:", y_val_encoded.shape)
print("Shape of y_test after one-hot encoding:", y_test_encoded.shape)

Shape of y_train after one-hot encoding: (493, 7)
Shape of y_val after one-hot encoding: (105, 7)
Shape of y_test after one-hot encoding: (106, 8)
